In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL 

In [6]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo csv "ADMISSIONS"

In [7]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\ADMISSIONS.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1


In [8]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "ADMISSIONS" en la BD

In [9]:
sql_insert = ''' INSERT 
                   INTO  ADMISSIONS
                         (row_id, subject_id, hadm_id, admittime, dischtime, deathtime, admission_type, admission_location, 
                          discharge_location, insurance, language, religion, marital_status, ethnicity, edregtime, 
                          edouttime, diagnosis, hospital_expire_flag, has_chartevents_data)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    adt = df.loc[i, 'admittime']
    dit = df.loc[i, 'dischtime']
    det = df.loc[i, 'deathtime']
    atm = df.loc[i, 'admission_type']
    alc = df.loc[i, 'admission_location']
    dlc = df.loc[i, 'discharge_location']
    ins = df.loc[i, 'insurance']
    lan = df.loc[i, 'language']
    rel = df.loc[i, 'religion']
    mas = df.loc[i, 'marital_status']
    etn = df.loc[i, 'ethnicity']
    edr = df.loc[i, 'edregtime']
    edo = df.loc[i, 'edouttime']
    dig = df.loc[i, 'diagnosis']
    hef = int(df.loc[i, 'hospital_expire_flag'])
    hcd = int(df.loc[i, 'has_chartevents_data'])
    reg_s = (row, subj, hadm, adt, dit, det, atm, alc, dlc, ins, lan, rel, mas, etn, edr, edo, dig, hef, hcd)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [10]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [11]:
cursor.close()
con.close()